In [ ]:
from IPython.core.display import display, HTML, Markdown
from ipywidgets import Button, HBox, VBox, widgets, Layout
from tabulate import tabulate
import pandas as pd

In [2]:
#solver solo per non fare i calcoli a mano

def num_percorsi(env,r,c):
    m = []
    for i in range(0,r):
        m.append([0 for j in range(0,c)])
    hole = 0
    i=0
    j=0
    for i in range(0,r):
        m[i][0]=0
    for j in range(0,c):
        m[0][j]=0
    for i in range(0,r):
        for j in range(0,c):
            if i==1 and j==1:
                m[i][j]=1
            else:
                if str(env[i][j])==-1:
                    m[i][j]=-1
                else:
                    if m[i-1][j] == -1 and m[i][j-1] == -1:
                         m[i][j] =0                    
                    elif m[i-1][j] == -1:
                        m[i][j]= m[i][j-1];
                    elif m[i][j-1] == -1:
                        m[i][j]= m[i-1][j]
                    else:
                        m[i][j] = m[i-1][j] + m[i][j-1];
        
    return m;

def num_percorsi_reverse(env,r,c):
    m = []
    for i in range(0,r):
        m.append([0 for j in range(0,c)])
    hole = 0
    i=r-1
    j=c-1
    while i > 0:
        if str(env[i][c-1])==-1:
            hole=1
            m[i][c-1]=-1
        elif hole == 0:
            m[i][c-1] = 1
        else:
            m[i][c-1] = 0
        i-=1
    hole=0
    while j > 0:
        if str(env[r-1][j])==-1:
            hole=1
            m[r-1][j] = -1
        elif hole == 0:
            m[r-1][j] = 1
        else:
            m[r-1][j] = 0
        j-=1
    for i in range(r-2,0,-1):
        for j in range(c-2,0,-1):
                if str(env[i][j])==-1:
                    m[i][j]=-1
                else:
                    if m[i+1][j] == -1 and m[i][j+1] == -1:
                         m[i][j] =0                    
                    elif m[i+1][j] == -1:
                        m[i][j]= m[i][j+1]
                    elif m[i][j+1] == -1:
                        m[i][j]= m[i+1][j]
                    else:
                        #print(f"{m[i+1][j]}+{m[i][j+1]}")
                        m[i][j] = m[i+1][j] + m[i][j+1];
    return m
            


In [3]:
def evaluation_format(answ, pt_green,pt_red):
    pt_blue=0
    if pt_green!=0:
        pt_blue=pt_red-pt_green
        pt_red=0
    return f"{answ}. Totalizzeresti <span style='color:green'>[{pt_green} safe pt]</span>, \
                                    <span style='color:blue'>[{pt_blue} possible pt]</span>, \
                                    <span style='color:red'>[{pt_red} out of reach pt]</span>.<br>"

def check_num_pathss_to_goal(num_paths, row, col):
    for i in range (0,row-1):
        if num_paths[i][0]!=0:
            return evaluation_format("No", 0, 10)+f"Attenzione, i cammini devono partire dalla cella $(1,1)$ e pertanto $num\_paths\_to[${i}$][0] = 0$"
    for j in range (0,col-1):
        if num_paths[0][j]!=0:
            return evaluation_format("No", 0, 10)+f"Attenzione, i cammini devono partire dalla cella $(1,1)$ e pertanto $num\_paths\_to[0][${j}$] = 0$"
    for i in range(row-1,0,-1):
        for j in range (col-1,0,-1):
            num_paths_forgiving = num_paths.copy()
            num_paths_forgiving[1][1] = 1 
            if num_paths_forgiving[i][j]<0:
                if num_paths_forgiving[i-1][j]<0 and num_paths_forgiving[i][j-1]<0:
                    if num_paths_forgiving[i][j]!=num_paths_forgiving[i-1][j]+num_paths_forgiving[i][j-1]:
                        return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
                elif num_paths_forgiving[i-1][j]<0:
                    if num_paths_forgiving[i][j]!=num_paths_forgiving[i-1][j]:
                        return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
                elif num_paths_forgiving[i][j-1]<0:
                    if num_paths_forgiving[i][j]!=num_paths_forgiving[i][j-1]:
                        return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
                else:
                    if num_paths_forgiving[i][j]!=0:
                        return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
    return  evaluation_format("Si", 1, 10)+"Non riscontro particolari problemi della tua versione della matrice $num\_paths\_to$."

def check_num_pathss_to_start(num_paths, row, col):
    for i in range(1,row-1):
        for j in range (1,col-1):
            if num_paths[i][j]<0:
                if i==row-1 and j==col-1:
                    if num_paths[i][j]!=1:
                        return  evaluation_format("No", 0, 10)+"Il valore nella cella $(n+1,m+1)$ non è corretto"
                elif num_paths[i+1][j]<0 and num_paths[i][j+1]<0:
                    if num_paths[i][j]!=num_paths[i+1][j]+num_paths[i][j+1]:
                        return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
                elif num_paths[i+1][j]<0:
                    if num_paths[i][j]!=num_paths[i+1][j]:
                        return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
                elif num_paths[i][j+1]<0:
                    if num_paths[i][j]!=num_paths[i][j+1]:
                        return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
                else:
                    if num_paths[i][j]!=0:
                        return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
    return  evaluation_format("Si", 1, 10)+"La tua soluzione è coerente"
                
def check_value(name,table,row,col, value):
    submission=f"Hai inserito il valore {value}, nella tabella {name} in posizione ({row},{col}) è presente il valore {table[row][col]}"
    if table[row][col]==value:
        return evaluation_format("Si", 1, 10)+f"La tua soluzione è coerente con quanto hai inserito nella tabella {name}<br>"+submission
    else:
        return evaluation_format("No", 0, 10)+f"La tua soluzione è coerente con quanto hai inserito nella tabella {name}<br>"+submission
    
def check_value_from_tables(table1, table2, row, col, value):
    submission=f"Hai inserito il valore {value}, il valore atteso ricavato dalle tue tabelle è {table1[row][col]*table2[row][col]}"
    if table[row][col]==value:
        return evaluation_format("Si", 1, 10)+f"La tua soluzione è coerente con quanto hai inserito nelle tabelle<br>"+submission
    else:
        return evaluation_format("No", 0, 10)+f"La tua soluzione è coerente con quanto hai inserito nella tabelle<br>"+submission

## Esercizio \[60 pts\]
(campo minato con monete) Ricerca di cammini in una griglia rettangolare con celle proibite e gemme da raccogliere.

Si consideri la seguente tabella di numeri interi.

In [8]:
m,n = 8,9
table = [
         [0, 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
         [0,-1 , 2 , 3 , 1 , 1 , 1 , 0 , 0 ,-1 ],
         [0, 3 , 3 , 1 , 0 ,-1 ,-1 , 0 , 0 , 0 ],
         [0, 2 ,-1 , 0 ,-1 , 0 , 0 , 1 , 1 , 1 ],
         [0, 0 , 0 , 1 , 0 , 0 , 0 , 1 ,-1 , 0 ],
         [0, 0 , 0 ,-1 , 1 ,-1 , 1 , 0 , 0 , 0 ],
         [0, 0 , 1 , 1 , 1 , 0 , 3 ,-1 , 0 , 1 ],
         [0, 3 ,-1 , 0 , 1 , 2 , 0 , 0 , 1 , 0 ],
         [0, 2 , 1 , 2 , 1 , 2 , 1 , 2 , 1 , 0 ]
        ]

In [5]:
columns=["1","2","3","4","5","6","7","8","9"]
index=pd.Index(["0","A","B","C","D","E","F","G","H"])
df=pd.DataFrame(table,index=index)
df = df.iloc[1:,1:]

print(tabulate(df, headers=columns, tablefmt='fancy_grid'))
#display(HTML(df.to_html()))

╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │   1 │   2 │   3 │   4 │   5 │   6 │   7 │   8 │   9 │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│ A  │  -1 │   2 │   3 │   1 │   1 │   1 │   0 │   0 │  -1 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ B  │   3 │   3 │   1 │   0 │  -1 │  -1 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ C  │   2 │  -1 │   0 │  -1 │   0 │   0 │   1 │   1 │   1 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ D  │   0 │   0 │   1 │   0 │   0 │   0 │   1 │  -1 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ E  │   0 │   0 │  -1 │   1 │  -1 │   1 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ F  │   0 │   1 │   1 │   1 │   0 │   3 │  -1 │   0 │   1 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ G  │   3 │  -1 │   0 │   1 │   2 │   0 │   0 │   1 │   0 │
├────┼─────┼─────┼─────┼

Un robot, inizialmente situato nella cella $A1=(1,1)$, deve portarsi nella cella $H9=(8,9)$.
Le celle che riportano un numero negativo contengono una mina od altre trapole mortali, ed il robot deve evitarle. Ogni altra cella contiene il numero di monete rappresentato nella tabella.\
I movimenti base possibili sono il passo verso destra (ad esempio il primo passo potrebbe avvenire dalla cella $A1$ alla cella $A2$) ed il passo verso il basso (ad esempio, come unica altra alternativa per il primo passo il robot potrebbe posrtarsi quindi nella cella $B1$).\
Quante monete può raccogliere al massimo il robot in un percorso che vada dalla cella $A1$ alla cella $H9$?\
E quanti sono i percorsi che gli consentono di raccogliere un tale numero di monete?

__Richieste__:

1. __\[10 pts\]__ A mano o tramite un programma componi la matrice $num\_paths\_to$ di dimensione $(m+1)\times(n+1)$, nella cui cella $num\_paths\_to[i][j]$, per ogni $i = 0,..., m+1$ e $j = 0,..., n+1$, sia riposto il numero di cammini dalla cella $A1=(1,1)$ alla generica cella $(i,j)$.

In [14]:
num_paths_to_goal= [ [0]*(n+1) for _ in range(m+1) ]
#num_paths_to_goal=[
#    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
#    [0, 1, 1, 1, 1, 1, 1, 1, 1, 'X'], 
#    [0, 1, 2, 3, 4, 'X', 'X', 1, 2, 2], 
#    [0, 1, 'X', 3, 'X', 0, 0, 1, 3, 5], 
#    [0, 1, 1, 4, 4, 4, 4, 5, 'X', 5], 
#    [0, 1, 2, 'X', 4, 'X', 4, 9, 9, 14], 
#    [0, 1, 3, 3, 7, 7, 11, 'X', 9, 23], 
#    [0, 1, 'X', 3, 10, 17, 28, 28, 37, 60], 
#    [0, 1, 1, 4, 14, 31, 59, 87, 124, 184]
#]
num_paths_to_goal= num_percorsi(table,len(table),len(table[0]))


In [15]:
display(Markdown("<b>La tua versione attuale della matrice $num\_paths\_to$ è la seguente:</b>"))

<b>La tua versione attuale della matrice $num\_paths\_to$ è la seguente:</b>

In [16]:
df2=pd.DataFrame(num_paths_to_goal,index=index)
df2 = df2.iloc[1:,1:]
print(tabulate(df2, headers=columns, tablefmt='fancy_grid'))

╒════╤═════╤═════╤═════╤═════╤═════╤═════╤══════╤══════╤══════╕
│    │   1 │   2 │   3 │   4 │   5 │   6 │    7 │    8 │    9 │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪══════╪══════╪══════╡
│ A  │   1 │   1 │   1 │   1 │   1 │   1 │    1 │    1 │    1 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼──────┼──────┼──────┤
│ B  │   1 │   2 │   3 │   4 │   5 │   6 │    7 │    8 │    9 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼──────┼──────┼──────┤
│ C  │   1 │   3 │   6 │  10 │  15 │  21 │   28 │   36 │   45 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼──────┼──────┼──────┤
│ D  │   1 │   4 │  10 │  20 │  35 │  56 │   84 │  120 │  165 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼──────┼──────┼──────┤
│ E  │   1 │   5 │  15 │  35 │  70 │ 126 │  210 │  330 │  495 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼──────┼──────┼──────┤
│ F  │   1 │   6 │  21 │  56 │ 126 │ 252 │  462 │  792 │ 1287 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼──────┼──────┼──────┤
│ G  │   1 │   7 │  28 │  84 │ 210 │ 462

In [18]:
display(Markdown(check_num_paths_to_goal(num_paths_to_goal,len(num_paths_to_goal),len(num_paths_to_goal[0]))))

Si. Totalizzeresti <span style='color:green'>[1 safe pt]</span>,                                     <span style='color:blue'>[9 possible pt]</span>,                                     <span style='color:red'>[0 out of reach pt]</span>.<br>La tua soluzione è coerente

1. __\[10 pts\]__ Componi ora una matrice $num\_paths\_from$, di dimensione $(m+2)times(n+2)$, nella cui cella $num\_paths\_from[i][j]$, per ogni $i = 1,..., m+1$ e $j = 1,..., n+1$, sia riposto il numero di cammini dalla generica cella $(i,j)$ alla cella $H9=(8,9)$.

In [ ]:
num_paths_to_start=[
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                        [0, 184, 63, 36, 9, 9, 9, 9, 2, 'X'], 
                        [0, 121, 27, 27, 0, 'X', 'X', 7, 2, 1], 
                        [0, 94, 'X', 27, 'X', 29, 17, 5, 1, 1], 
                        [0, 94, 49, 27, 27, 12, 12, 4, 'X', 1], 
                        [0, 45, 22, 'X', 15, 'X', 8, 4, 4, 1], 
                        [0, 23, 22, 22, 15, 9, 4, 'X', 3, 1], 
                        [0, 1, 'X', 7, 6, 5, 4, 3, 2, 1], 
                        [0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
                    ]
#num_paths_to_start=num_percorsi_reverse(table,len(table),len(table[0]))

In [ ]:

df3=pd.DataFrame(num_paths_to_start,index=index)
df3 = df3.iloc[1:,1:]
print(tabulate(df3, headers=columns, tablefmt='fancy_grid'))

In [ ]:
display(Markdown(check_num_paths_to_start(num_paths_to_start,len(num_paths_to_start),len(num_paths_to_start[0]))))

1. __\[10 pts\]__ [Esercizio senza monete] Quanti sono i percorsi possibili se la partenza è in A–1?

In [ ]:
ans1 = 184

In [ ]:
display(Markdown(check_value("num_paths_to_start",num_paths_to_start,1,1,ans1)))

2. __\[10 pts\]__ E se la partenza è in B–3?

In [ ]:
ans2=27

In [ ]:
display(Markdown(check_value("num_paths_to_start",num_paths_to_start,2,3,ans2)))

3. __\[10 pts\]__ E se con partenza in A–1 il robot deve giungere in F–6?

In [ ]:
ans3=11

In [ ]:
display(Markdown(check_value("num_paths_to_goal",num_paths_to_goal,6,6,ans3)))

4. __\[10 pts\]__ E se con partenza in A–1 ed arrivo in H–9 al robot viene richiesto di passare per
la cella D–5?

In [ ]:
ans4=0

In [ ]:
display(Markdown(check_value_from_tables(num_paths_to_start,num_paths_to_goal,4,5,ans4)))

5. __\[10 pts\]__ Quale é il massimo valore in premi raccoglibili lungo una traversata da A–1 a H–9?

6. __\[10 pts\]__ Quanti sono i percorsi possibili che assicurino di portare a case tale massimo valore?